In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [3]:
# run optimization with constraints
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

/Users/helangchun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [4]:
result.params

[0.7960795534069122,
 0.3516828940512714,
 -0.1876339114390952,
 0.3967686699928462,
 array([-0.92567808,  0.14851819])]

In [5]:
result.gamma()

-0.1876339114390952

In [6]:
result.residual_mean()

-0.17456338672509997

In [7]:
result.residual_std()

0.7230924663053737

In [8]:
result.asymp_var()

array([[ 0.14642534,  0.11046095, -0.04967592, -0.06673693, -0.00620082,
        -0.00388322],
       [ 0.11046095,  0.22903132, -0.10676837, -0.04390049,  0.0085739 ,
         0.00131084],
       [-0.04967592, -0.10676837,  0.14581486,  0.03198589, -0.03174064,
        -0.00954728],
       [-0.06673693, -0.04390049,  0.03198589,  0.09327817, -0.01633044,
        -0.0044735 ],
       [-0.00620082,  0.0085739 , -0.03174064, -0.01633044,  0.10031501,
         0.03197351],
       [-0.00388322,  0.00131084, -0.00954728, -0.0044735 ,  0.03197351,
         0.06822769]])

In [9]:
result.print_table()

Residuals:
     Min         1Q     Median       3Q       Max
--------  ---------  ---------  -------  --------
-1.83015  -0.418611  0.0511063  0.29274  0.802542

Estimated Parameters:
  Sigma    Lambda      Gamma       Rho  Beta
-------  --------  ---------  --------  -------------------------
0.79608  0.351683  -0.187634  0.396769  [-0.92567808  0.14851819]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1     alpha_2    alpha_3
---------  ---------  ----------  ---------
 0.258994  -0.412609  -0.0845264   0.238142

Estimated c0 (display first 10 rows): 
     c0_0      c0_1       c0_2      c0_3
---------  --------  ---------  --------
-0.199493  0.734334  -0.265818  0.152097

Estimated asympotitic variance: 
               Gamma          Rho       Beta_0      Beta_1       Lambda      Sigma^2
-------  -----------  -----------  -----------  ----------  -----------  -----------
Gamma     0.146425     0.110461    -0.0496759   -0.0667369  -0.00620082  -0.00388322
Rho       0.

In [11]:
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, constrain= False)

In [12]:
result.print_table()

Residuals:
     Min         1Q    Median        3Q      Max
--------  ---------  --------  --------  -------
-1.53149  -0.638114   0.11643  0.686326  1.35756

Estimated Parameters:
  Sigma    Lambda      Gamma        Rho  Beta
-------  --------  ---------  ---------  -------------------------
1.09333  0.983577  -0.696034  0.0881744  [-1.07835509  0.34635336]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
 0.708948   -1.03964  0.0618047   0.268884

Estimated c0 (display first 10 rows): 
     c0_0      c0_1       c0_2       c0_3
---------  --------  ---------  ---------
-0.489306  0.608697  -0.889519  -0.206171

Estimated asympotitic variance: 
               Gamma          Rho      Beta_0      Beta_1      Lambda      Sigma^2
-------  -----------  -----------  ----------  ----------  ----------  -----------
Gamma     0.287617     0.218237    -0.0922129  -0.129178   -0.0172564  -0.00320748
Rho       0.218237

### 2. Test k=1

In [13]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 20
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [14]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [15]:
result.print_table()

Residuals:
     Min        1Q     Median        3Q      Max
--------  --------  ---------  --------  -------
-4.20873  -1.40484  -0.611157  0.270158  2.64376

Estimated Parameters:
  Sigma    Lambda      Gamma       Rho     Beta
-------  --------  ---------  --------  -------
1.01538  0.503294  0.0745974  0.215233  1.15498

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3     alpha_4    alpha_5     alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ----------  ---------  ----------  ---------  ---------  ---------
 -0.13756   0.437013  -0.294963   0.393413  -0.0330562  -0.698087  -0.0655998  0.0522773    1.46529   -1.11872

Estimated c0 (display first 10 rows): 
     c0_0     c0_1       c0_2       c0_3       c0_4      c0_5      c0_6      c0_7       c0_8       c0_9
---------  -------  ---------  ---------  ---------  --------  --------  --------  ---------  ---------
-0.354209  1.39985  0.0387434  -0.387587  -0.177494

In [16]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [17]:
result.print_table()

Residuals:
     Min        1Q     Median        3Q      Max
--------  --------  ---------  --------  -------
-4.20873  -1.40484  -0.611157  0.270158  2.64376

Estimated Parameters:
  Sigma    Lambda      Gamma       Rho     Beta
-------  --------  ---------  --------  -------
1.01538  0.503294  0.0745974  0.215233  1.15498

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3     alpha_4    alpha_5     alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ----------  ---------  ----------  ---------  ---------  ---------
 -0.13756   0.437013  -0.294963   0.393413  -0.0330562  -0.698087  -0.0655998  0.0522773    1.46529   -1.11872

Estimated c0 (display first 10 rows): 
     c0_0     c0_1       c0_2       c0_3       c0_4      c0_5      c0_6      c0_7       c0_8       c0_9
---------  -------  ---------  ---------  ---------  --------  --------  --------  ---------  ---------
-0.354209  1.39985  0.0387434  -0.387587  -0.177494